In [170]:
# Dependencies - required modules
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import lxml.html as lh
import time

In [171]:

    # NASA Mars News


In [172]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?&blank_scope=Latest'
time.sleep(1)

In [173]:
# Retrieve page with the requests module  - #http get request to the url
response = requests.get(url)
time.sleep(1)

In [174]:
# Create BeautifulSoup object; parse with 'lxml
soup = BeautifulSoup(response.text, 'lxml')
time.sleep(1)

In [175]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [176]:
# Retrieve the title and paragraph for all articles - results are returned as an iterable list
res = soup.find('div', class_='image_and_description_container')
time.sleep(1)

In [177]:
#results

# scrape the article title 
news_title = res.find_next(class_="content_title").text

# scrape the article paragraph
news_p = res.find(class_="rollover_description_inner").text

# print article data
print('-----------------')
print(news_title)
print(news_p)
time.sleep(1)

-----------------


NASA to Host Virtual Briefing on February Perseverance Mars Rover Landing



NASA leadership and members of the mission will discuss the agency’s latest rover, which touches down on the Red Planet on Feb. 18.



In [178]:
# create mars_data dict that we can insert into mongo
mars_data = {}

# add our news data to the mars dictionary
mars_data["news_title"] = news_title
mars_data["news_p"] = news_p

In [179]:

    # JPL Mars Space Images - Featured Image
    

In [180]:
# Setup splinter 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
time.sleep(3)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [C:\Users\chuck\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


In [181]:
# Link up to the jet propulsion labs nasa web site
url = 'https://www.jpl.nasa.gov/'
browser.visit(url)
time.sleep(1)

In [182]:
#Find and click on the images button
browser.links.find_by_partial_text('Images').click()
time.sleep(1)

#Select Mars filter
browser.find_by_value('Mars').click()
time.sleep(1)



In [183]:
# Now that splinter has us on the right page, we use beautiful soup to find and isolate the first mars image

url = 'https://www.jpl.nasa.gov/images'


html = browser.html
# Create BeautifulSoup object; parse with 'html
soup = BeautifulSoup(html, 'html.parser')



In [184]:
# Retrieve the cover object for the first image - results are returned as an iterable list
image = soup.find('div', class_='sm:object-cover object-cover')

#results
# scrape the first mars web link 
featured_image_url = image.find("img")['data-src']

featured_image_url


'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24405.2e16d0ba.fill-400x400-c50.jpg'

In [185]:
#Quit Browswer
browser.quit()

In [186]:
# add our featured image to the mars dictionary
mars_data["featured_image"] = featured_image_url

In [187]:

    # Mars Facts


In [188]:
url='https://space-facts.com/mars'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [189]:
#Check the length of the rows, to ensure that all the rows have the same width
[len(T) for T in tr_elements[:9]]

[2, 2, 2, 2, 2, 2, 2, 2, 2]

In [190]:
#Pull the record detail from the table

#Define the elements
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[:9]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name))

1:"Equatorial Diameter:6,792 km"
2:"Polar Diameter:6,752 km"
3:"Mass:6.39 × 10^23 kg (0.11 Earths)"
4:"Moons:2 (Phobos & Deimos)"
5:"Orbit Distance:227,943,824 km (1.38 AU)"
6:"Orbit Period:687 days (1.9 years)"
7:"Surface Temperature: -87 to -5 °C"
8:"First Record:2nd millennium BC"
9:"Recorded By:Egyptian astronomers"


In [191]:
#Create Pandas DataFrame

df=pd.DataFrame(col, columns = ["Trait"])    

df[['Trait','Measures']] = df['Trait'].str.split(':',expand=True)

df

,Trait,Measures
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [192]:
mars_facts = df.to_dict('records')

In [193]:
#Set Trait column as the index
df.set_index('Trait', inplace=True)

In [194]:
#Convert dataframe to an html file
mars_facts_html = df.to_html(classes="table table-striped table-bordered")
df.to_html(open('mars_facts.html', 'w'))

In [195]:
# add our mars facts to the mars dictionary
mars_data["mars_facts"] = mars_facts_html

In [196]:
mars_facts_html

'<table border="1" class="dataframe table table-striped table-bordered">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measures</th>\n    </tr>\n    <tr>\n      <th>Trait</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By</th>\n      <td>Egyptian astro

In [197]:
    # Mars hemispheres

In [198]:
#Use a Python dictionary to store the Mars Hemisphers data

hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg"},
    {"title": "Cerberus Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg"},
    {"title": "Schiaparelli Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/55f04ff759b242bdff8833374544b1be_syrtis_major_unenhanced.tif_full.jpg"},
    {"title": "Syrtis Major Hemisphere", "img_url": "https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg"}
]


In [199]:
# add our mars hemispheres to the mars dictionary
mars_data["mars_hemispheres"] = hemisphere_image_urls

mars_data

{'news_title': '\n\nNASA to Host Virtual Briefing on February Perseverance Mars Rover Landing\n\n',
 'news_p': '\nNASA leadership and members of the mission will discuss the agency’s latest rover, which touches down on the Red Planet on Feb. 18.\n',
 'featured_image': 'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24405.2e16d0ba.fill-400x400-c50.jpg',
 'mars_facts': '<table border="1" class="dataframe table table-striped table-bordered">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Measures</th>\n    </tr>\n    <tr>\n      <th>Trait</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance</t